## SCF参数修改

**环境**：
- Python 3.5+
- lxml 3.7.3

**功能**
- 用预定义参数字典中的参数值替换原SCF文件中对应的参数值并生成新的SCF文件

**限制**
- 不支持对同一个节点下多个同名的参数的修改，比如qcitab/pdcp profile/rlc profile下参数

**历史**
- 0.01: 2017/08/07

**TO DO**

In [1]:
# 参数字典
para_dict = {
    # Basic Info
#     'chBw':'20MHz', 'earfcnDL':'1725', 'earfcnUL':'19725', 'earfcn':'38968','tddFrameConf':'2', 'tddSpecSubfConf':'7', 
    'inactivityTimer':'3600', 'maxNrSymPdcch':'3', 
    'actModulationSchemeDl':'64QAM', 'actModulationSchemeUl':'16QAMHighMCS',
    # MIMO & RI
    'dlMimoMode':'Dynamic Open Loop MIMO', 'syncSigTxMode':'TxDiv', 'riEnable':'true', 'riPerM':'1', 'riPerOffset':'-1',
    # Capacity
    'maxNumActUE':'400', 'maxNumActDrb':'1200', 'maxNumRrc':'400', 'maxNumRrcEmergency':'400',
    'maxNumUeDl':'12', 'maxNumUeUl':'12', 'maxNumUeDlDwPTS':'8', 'nCqiDtx':'0',
    'cellSrPeriod':'80ms', 'cqiPerNp':'40ms', 'n1PucchAn':'36', 'nCqiRb':'3', #PUCCH format 2/2b
    'deltaPucchShift':'1','pucchNAnCs':'0', #PUCCH 1/1a/1b
    'nPucchF3Prbs':'0', #PUCCH format 3 
    'prachFreqOff':'4',
    # Feature
    'actDrx':'true','actSmartDrx':'false', 'actAutoPucchAlloc':'true',
    # CA
    'actDLCAggr':'true', 'actULCAggr':'false', 'actInterEnbDLCAggr':'false', 
    'maxNumScells':'2','maxNumCaConfUe':'100', 'maxNumCaConfUeDc':'100', 'maxNumCaConfUe3c':'20',
    # SRS
    'srsActivation':'false','srsBwConf':'0bw','srsOnTwoSymUpPts':'true','srsSubfrConf':'sc4','beamformingType':'nonBeamforming',
    'srsBandwidth':'3hbw','srsUePeriodicity':'10ms',
    # Others
    'recoveryResetDelay':'10000'
}

from lxml import etree
import os

file_path = r"D:\userdata\anliu\Desktop" 
in_file = 'Commissioning_BTS140_20170803_IB2.xml'
out_file = 'Commissioning_BTS140_20170803_IB2_new.xml'

os.chdir(file_path)

def replace_para_value(p_node, para_dict):
    # 如果p_node下有para_dict中的参数，该标志位置为True
    is_present = False
    # 查找p_node下参数字典中的参数    
    for key, value in para_dict.items():
        # 参数列表中的参数只可能在某个节点下出现一次，所以只需要使用find而不是findall
        node = p_node.find(".//*[@name='%s']" % key)
        # 参数字典中参数可能出现在同一个p_node下，如果p_node下找到参数列表中参数，打印且仅打印一次p_node的attrib
        if node != None and is_present == False:
            is_present = True
            print('='*12 + p_node.attrib['distName'] + '='*12)
        elif node == None:
            continue
        # 打印参数列表参数值得变化
        node_old_text = node.text
        node.text = para_dict[key]
        print(key + ': ' + node_old_text + ' -> ' + node.text)  

# 读取scf文件并获得根节点            
tree = etree.parse(in_file)
root = tree.getroot()
# replace_para_value(root, para_dict) 

# scf文件中需要修改的参数只可能存在于包含属性distName的节点下，所以首先需要找出包含属性distName的节点
node_list = root.findall(".//*[@distName]")
# 遍历每一个包含distName属性的节点，替换参数字典中的值。
for p_node in node_list:
    replace_para_value(p_node, para_dict)

# 参数值修改后写入新的scf文件
tree.write(out_file, encoding='utf-8', xml_declaration=True) 

============MRBTS-140/LNBTS-140============
recoveryResetDelay: 10000 -> 10000
actDLCAggr: false -> true
actInterEnbDLCAggr: false -> false
actULCAggr: false -> false
nCqiDtx: 0 -> 0
============MRBTS-140/LNBTS-140/LNCEL-11============
maxNumScells: 1 -> 2
actDrx: false -> true
actModulationSchemeDl: QPSK -> 64QAM
actSmartDrx: false -> false
inactivityTimer: 30 -> 3600
============MRBTS-140/LNBTS-140/LNCEL-12============
maxNumScells: 1 -> 2
actDrx: false -> true
actModulationSchemeDl: QPSK -> 64QAM
actSmartDrx: false -> false
inactivityTimer: 30 -> 3600
============MRBTS-140/LNBTS-140/LNCEL-13============
maxNumScells: 1 -> 2
actDrx: true -> true
actModulationSchemeDl: 64QAM -> 64QAM
actSmartDrx: false -> false
inactivityTimer: 36000 -> 3600
============MRBTS-140/LNBTS-140/LNCEL-14============
maxNumScells: 1 -> 2
actDrx: true -> true
actModulationSchemeDl: 64QAM -> 64QAM
actSmartDrx: false -> false
inactivityTimer: 36000 -> 3600
============MRBTS-140/LNBTS-140/LNCEL-11/LNCEL_FDD-0===